In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

from tqdm import tqdm
from pathlib import Path

import torch

from utils.data import get_hsm_dataset, get_solar_energy_dataset, get_fuel_prices_dataset, get_passengers_dataset, split_data
from utils.metrics import MAPE, WAPE, MAE
from utils.dl import QuantGAN_Discriminator, QuantGAN_Generator
from utils.QuantGAN_gaussianize import Gaussianize

In [2]:
hsm_dataset_path = Path("data/huge_stock_market_dataset/")
solar_energy_dataset_path = Path("data/solar_energy/")
fuel_prices_dataset_path = Path("data/fuel_prices/")
passengers_dataset_path = Path("data/air_passengers/")
models_dir = Path("models/")

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

batches_to_gen = 1

num_epochs = 10
nz = 3
batch_size = 80
seq_len = 127
clip = 0.01
lr = 0.0002
receptive_field_size = 127

cuda:0


In [4]:
class Loader32(torch.utils.data.Dataset):
    def __init__(self, data, length):
        assert len(data) >= length
        self.data = data
        self.length = length
    
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx:idx+self.length]).reshape(- 1, self.length).to(torch.float32)
        
    def __len__(self):
        return max(len(self.data)-self.length, 0)

In [5]:
def model_routine(time_series, ts_index, synthetic_path):
    global val_size, test_size, batches_to_gen, num_epochs, nz, batch_size, seq_len, clip, lr, receptive_field_size
    
    # preprocessing steps according to the QuanGAN paper
    df = time_series
    # returns = df.shift(1) / df - 1
    # log_returns = np.log(df / df.shift(1))[1:].to_numpy().reshape(- 1, 1)
    log_returns = df.values.reshape(- 1, 1)
    standardScaler1 = StandardScaler()
    standardScaler2 = StandardScaler()
    gaussianize = Gaussianize()
    log_returns_preprocessed = standardScaler2.fit_transform(gaussianize.fit_transform(standardScaler1.fit_transform(log_returns)))
    data_size = log_returns.shape[0]

    # defining models and optimizers
    generator = QuantGAN_Generator().to(device)
    discriminator = QuantGAN_Discriminator(seq_len).to(device)
    disc_optimizer = torch.optim.RMSprop(discriminator.parameters(), lr=lr)
    gen_optimizer = torch.optim.RMSprop(generator.parameters(), lr=lr)

    # data preparing
    dataset = Loader32(log_returns_preprocessed, receptive_field_size)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
    
    t = tqdm(range(num_epochs))
    for epoch in t:
        for idx, data in enumerate(dataloader, 0):

            discriminator.zero_grad()
            real = data.to(device)
            batch_size, seq_len = real.size(0), real.size(2)
            noise = torch.randn(batch_size, nz, seq_len, device=device)
            fake = generator(noise).detach()
            disc_loss = - torch.mean(discriminator(real)) + torch.mean(discriminator(fake))
            disc_loss.backward()
            disc_optimizer.step()

            for dp in discriminator.parameters():
                dp.data.clamp_(-clip, clip)
    
            if idx % 5 == 0:
                generator.zero_grad()
                gen_loss = - torch.mean(discriminator(generator(noise)))
                gen_loss.backward()
                gen_optimizer.step()
        t.set_description('Discriminator Loss: %.8f Generator Loss: %.8f' % (disc_loss.item(), gen_loss.item()))
    # saving model
    torch.save(generator, models_dir /  f'QuantGAN_generator_selected{ts_index}.pth')

    # generation synthetic time series
    generator.eval()
    ys = []
    for _ in range(batches_to_gen):
        with torch.no_grad():
            noise = torch.randn(80, 3, 127).to(device)
            y = generator(noise).cpu().detach().squeeze()

        y = (y - y.mean(axis=0)) / y.std(axis=0)
        y = standardScaler2.inverse_transform(y)
        y = np.array([gaussianize.inverse_transform(np.expand_dims(x, 1)) for x in y]).squeeze()
        y = standardScaler1.inverse_transform(y)

        # some basic filtering to redue the tendency of GAN to produce extreme returns
        # y = y[(y.max(axis=1) <= 2 * log_returns.max()) & (y.min(axis=1) >= 2 * log_returns.min())]
        # y -= y.mean()
        ys.append(y)

    np.save(synthetic_path / f"selected{ts_index}.npy", np.row_stack(ys))

    del discriminator, generator, disc_loss, gen_loss, dataloader, dataset, y
    torch.cuda.empty_cache()

In [6]:
ts_iterator = get_hsm_dataset(hsm_dataset_path, selected_files=hsm_dataset_path / "selected100.csv")
synthetic_path = hsm_dataset_path / "synthetic/QuantGAN/"
start_point = 100
for _ in range(start_point): next(ts_iterator)

for ts_index, time_series in enumerate(ts_iterator, start_point):
    print(f"Time Series #{ts_index}")
    model_routine(time_series, ts_index, synthetic_path)

In [7]:
num_epochs = 4
ts_iterator = get_solar_energy_dataset(solar_energy_dataset_path, max_results=10)
synthetic_path = solar_energy_dataset_path / "synthetic/QuantGAN/"
start_point = 10
for _ in range(start_point): next(ts_iterator)

for ts_index, time_series in enumerate(ts_iterator, start_point):
    print(f"Time Series #{ts_index}")
    model_routine(time_series, ts_index, synthetic_path)

In [8]:
num_epochs = 10
ts_iterator = get_fuel_prices_dataset(fuel_prices_dataset_path)
synthetic_path = fuel_prices_dataset_path / "synthetic/QuantGAN/"
start_point = 8
for _ in range(start_point): next(ts_iterator)

for ts_index, time_series in enumerate(ts_iterator, start_point):
    print(f"Time Series #{ts_index}")
    model_routine(time_series, ts_index, synthetic_path)

In [9]:
num_epochs = 4
ts_iterator = get_passengers_dataset(passengers_dataset_path, max_results=99)
synthetic_path = passengers_dataset_path / "synthetic/QuantGAN/"
start_point = 99
for _ in range(start_point): next(ts_iterator)

for ts_index, time_series in enumerate(ts_iterator, start_point):
    print(f"Time Series #{ts_index}")
    model_routine(time_series, ts_index, synthetic_path)

# Similarity

In [10]:
from tqdm import tqdm
from pathlib import Path

import numpy as np
import pandas as pd
from scipy.special import kl_div

from utils.data import get_hsm_dataset, get_solar_energy_dataset, get_fuel_prices_dataset, get_passengers_dataset, split_data, log_returns
from utils.synth_eval import eval_sim

In [11]:
results_dir = Path("results")
seq_len = 127
hsm_dataset_path, solar_energy_dataset_path, fuel_prices_dataset_path, passengers_dataset_path = [Path(x) for x in (hsm_dataset_path, solar_energy_dataset_path, fuel_prices_dataset_path, passengers_dataset_path)]

In [12]:
eval_sim(("hsm", "se", "fp", "ap"), (hsm_dataset_path, solar_energy_dataset_path, fuel_prices_dataset_path, passengers_dataset_path),
     "QuantGAN", save=True, results_dir=results_dir)

processing hsm dataset


100it [00:14,  6.91it/s]


processing se dataset


0it [00:00, ?it/s]c:\Users\Владислав\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
1it [00:00,  1.09it/s]c:\Users\Владислав\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
2it [00:01,  1.11it/s]c:\Users\Владислав\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
3it [00:02,  1.12it/s]c:\Users\Владислав\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
4it [00:03,  1.11it/s]c:\Users\Владислав\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\cor

processing fp dataset


8it [00:00, 13.43it/s]


processing ap dataset


3it [00:12,  2.79s/it]c:\Users\Владислав\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
4it [00:12,  1.82s/it]c:\Users\Владислав\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
6it [00:12,  1.05it/s]c:\Users\Владислав\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
19it [00:17,  2.95it/s]c:\Users\Владислав\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
28it [00:20,  3.00it/s]c:\Users\Владислав\AppData\Local\Programs\Python\Python38\lib\site-packages\pand

defaultdict(dict,
            {'hsm': {'kl_div': inf, 'kstest_pval': 3.703076513218439e-31},
             'se': {'kl_div': nan, 'kstest_pval': 1.3039956366633642e-20},
             'fp': {'kl_div': inf, 'kstest_pval': 1.3813869469290808e-75},
             'ap': {'kl_div': nan, 'kstest_pval': 8.354717247922456e-35}})